# In this exercize, we'll see an example application of how we can use tf-idf vectorization to create a simple search engine 

### start by importing the necessary libraries

In [1]:
import pandas as pd
# we use the pre-made tfidf vectorization algorithm from nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# we use cosine similarity as our measure to infer which text is more similar to our search query.
# You may experiment with different distance measurement algorithms, like eucledian distance,
# and check which algorithm performs better in this application.
# reference: https://scikit-learn.org/stable/modules/classes.html#pairwise-metrics
from sklearn.metrics.pairwise import cosine_similarity

### Load the tweets

In [2]:
df = pd.read_csv('tweets.csv')

df.head()

,Unnamed: 0,date,id,link,retweet,text,author
0,0,Oct 7,784609194234306560,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump
1,1,Oct 10,785608815962099712,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.c...,DonaldTrump
2,2,Oct 8,784840992734064640,/realDonaldTrump/status/784840992734064641,False,The media and establishment want me out of the...,DonaldTrump
3,3,Oct 8,784767399442653184,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump
4,4,Oct 10,785561269571026944,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\n#MAGA #A...,DonaldTrump


### Initialize our tf-idf vectorizer, and vectorize our tweets

In [4]:
# remove unnecessary characters, perform regex parsing
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
def filter(text):
    # remove punctuation and special characters
    text = re.sub("\[.,\/#!$%\^&\*;:{}=\-_`~()]","",text)
    # replace newline with space
    text = re.sub("[\\t\\n\\r]+"," ",text)
    return text

#Tokenize and perform lemmatization
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    return ''

def clean(text):
    text = filter(text)
    # tokenize
    text = word_tokenize(text)
    # only alphabets and numerics and lower case
    text = [word.lower() for word in text if word.isalpha()]
    # apply lemmatization
    text = [word for word in text if not word in stopwords.words('english')]

    output = []

    for i, word in enumerate(text):
        pos = get_wordnet_pos(pos_tag([word])[0][1])
        if pos != '':
            output.append(lemmatizer.lemmatize(word, pos))
        else:
            output.append(word)
    return output

In [7]:
# Create a TFIDF Vectorizer
tfidf_ = TfidfVectorizer(tokenizer=clean, ngram_range=(1, 1), strip_accents='ascii') # ngram_range combinaisons de mots : (1,4) 1 mot et jusqu'à 4 mots


In [8]:
# Use the vectorizer's fit_transform function
corpus = tfidf_.fit_transform(df.text) # <=> df['text']
corpus = pd.DataFrame(corpus.todense(), columns=tfidf_.get_feature_names())

print("\nFinal tf-idf vectorizer matrix form :\n")
corpus.head()


Final tf-idf vectorizer matrix form :



,aaa,aaanews,aandreakis,aaszkler,ab,abandon,abanoubabdel,abbeyashir,abbott,abbottabad,...,zkoppe,zogby,zone,zoom,zrill,zucker,zuckerberg,zuckerman,zuker,zwlykins
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
search_query = "Take it easy with me, please"
search_query = pd.Series(search_query)

In [10]:
tweet = tfidf_.transform(search_query)

In [11]:
tweet = pd.DataFrame(tweet.todense(), columns=tfidf_.get_feature_names())

In [12]:
pd.options.display.max_columns = None
tweet.head()

,aaa,aaanews,aandreakis,aaszkler,ab,abandon,abanoubabdel,abbeyashir,abbott,abbottabad,abbysullz,abc,abcinsc,abcnews,abcpolitics,abcsharktank,abcwashington,abdeslam,abdullah,abedin,abedini,abedinis,abelerner,aberdeen,aberdeencc,aberdeenshire,aberdonian,abhor,abide,ability,abimlebt,able,abnormally,aboard,abolish,abouttherock,abraham,abridge,abroad,abrupt,absabella,absence,absolute,absoluteicon,absolutely,absorb,absorbed,abt,abu,abunnieslife,abuse,abuser,abusive,abysmal,ac,academic,academy,accelerate,accentuate,accept,acceptable,acceptance,accepted,accepts,access,accesshollywood,accessible,accessory,accident,accidental,accidentally,acclaim,accolade,accompany,accomplish,accomplishment,accord,accordingly,account,accountability,accountable,accrue,acct,accumulate,accuracy,accurate,accurateif,accuse,ace,acebuey,acewill,ache,acheson,achieve,achievement,achiever,achieves,achilarre,acknowledge,acknowledgement,aclj,acn,acnnews,acquire,acquisition,acquit,acre,across,act,action,activation,active,actively,activist,activity,actor,actress,actual,actually,actuallydominic,acu,acumen,ad,adam,adambrownagency,adamcarolla,adammatruski,adamogdenceo,adamscott,adamscottnow,adamsteinbaugh,adapt,adaptable,add,addict,addiction,addition,additional,additionally,address,adelson,adios,adiosamerica,adjunct,adjust,adkradio,adm,admin,administer,administration,admirable,admiral,admiration,admire,admit,admits,admonish,ado,adopt,adorable,adore,adores,adrian,adriandemers,adsdesperate,adsthe,adult,adultprodigy,advance,advanced,advantage,advantageous,advantagesmart,adventure,adversary,adverse,adversity,advertina,advertisement,advertiser,advertising,advice,advise,adviser,advisor,advisorsource,advisory,advocate,aerial,aerosmith,aesop,aesthetic,aetv,aews,affair,affect,affected,afford,affordable,afghan,afghani,afghanistan,afghanistant,afpfnh,afraid,africa,africabefore,african,afrodeziak,afternoon,aftershave,afterwards,ag,agaiin,againeverybody,againhttp,age,agency,agenda,agent,agentsanderson,aggravation,aggression,aggressive,aggressively,aggressor,agillogly,agnstt,ago,agoa,agoraindex,agree,agreement,agrees,agriculture,agschneiderman,agschneidermans,agseliza,ahataas,ahatwearer,ahead,ahhh,ahhhhhh,ahmadinejad,ahmadinejads,ai,aid,aide,aidforafrica,aig,ailes,ailsa,aim,aipac,air,airbase,airbus,aircraft,airline,airman,airplane,airport,airspace,airstrikes,airtime,airwave,airway,aj,ajdukakis,ajfrugoli,aka,akbar,akmumpreneur,akoya,akron,al,alabama,alabamaftbl,alamsyahdoni,alan,alancolmes,alannawannabe,alarmist,alaska,albany,albeit,albert,album,albusinvogue,alcohol,aldous,aldridge,alec,aleksmm,alerdzcuevas,alert,alertserial,alex,alexander,alexandracmauro,alexbrochesmtv,alexeveslage,alexiskrystina,alexlevandoski,alexlyonnn,alexmaymcgregor,alexpappas,alexperrry,alexrasey,alexsalmond,alexsalmonds,alexsiminoff,alexskoog,alextallitsch,algae,algemeiner,algeria,algore,ali,alien,alike,alinsky,alinskyhttp,alisa,alison,alisyn,alisyncamerota,alive,alivelutheran,aljazeera,allah,allby,allege,allegiance,alleliua,allen,allenwest,allenwronowski,alleras,alliance,allibiis,allis,alllllll,allow,allowedonly,allows,allred,allrepublicans,allstargame,alltheway,ally,allyn,allysonwardbrkr,alma,almost,aloha,alone,along,alongside,aloracai,alot,alphainparis,alphaquintesson,alphatreblesix,already,alright,alsharpton,alsicebucketchallenge,also,alternative,although,altogether,alumnus,alvisefranz,alwaleedbint,always,alwaystrump,alyssa,alyssamenasci,alyssathgreat,ama,amadorserver,amanda,amandajocountry,amandaknox,amandamichl,amandatmiller,amaninmanhattan,amarudacosta,amass,amateur,amaze,amazes,amazingly,amazingthe,amazinhof,amazon,ambassador,ambassadorrice,ambassadorship,ambassadorshiphttp,ambeylambey,ambjohnbolton,amc,amelia,amen,amendment,amenity,amercia,america,americafirst,americafirsthttp,americafirsthttps,americahttp,american,americanaspie,americanexpress,americanhowl,americanism,americanvalues,americashe,ames,amesmoreno,amherst,amid,amiguiza,amira,amitchellmo,ammo,ammunition,amnesty,amnestyhttp,amoakabr,among,a

### Vectorize our search query using the same vectorizer that we've used to vectorize the tweets, and calculate the cosine similarity between it and all of our vectorized tweets 

In [13]:
similarity = cosine_similarity(corpus, tweet).reshape((-1,))

### Sort the results in descending order of similarity with the search query

In [14]:
similarity.argsort()[:-10][::-1]

array([ 6370,  4209,  6450, ..., 11359, 11358,     0], dtype=int64)

In [15]:
#get the most similar tweet text with the search query
df.iloc[6370,5]

'"@fredsmith433: @realDonaldTrump Please run for president"'